# Fair Binary Classification with SearchFair on CelebA and Adult

Here, we show how to use SearchFair on two datasets: CelebA and Adult

## Imports

We start by importing SearchFair from the installed package.

In [1]:
from searchfair import SearchFair

Second, we load some necessary methods and numpy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

# The optimization does not always comply with the new cvxpy dpp disciplined programming rules. 
# but this is not a problem. 
import warnings
warnings.filterwarnings('ignore')

# The CelebA dataset

On the Celebrity Faces dataset we are given descriptions of celebrity faces, with 40 binary attributes. Here, we use the Attribute 'Smiling' as the class label, and sex as the sensitive attribute. 

In [3]:
import get_real_data as get_data

# Load Data
x_data, y_data, s_data = get_data.get_celebA_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1200
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

Here are some basic information about the dataset:

In [4]:
import utils as ut
ut.print_data_stats(s_data, y_data)

Total data points: 202599
# non-protected examples: 118165
# protected examples: 84434
# non-protected examples in positive class: 63871 (54.1%)
# protected examples in positive class: 33798 (40.0%)


## Creating a SearchFair Model

### Demographic Parity

To learn a classifier with SearchFair, we need to choose a kernel between 'linear' and 'rbf', and we need to choose a fairness notion - either Demographic Parity (DDP) or Equality of Opportunity (DEO). 

In [5]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress

Let us choose a regularization parameter and then fit a model with the default settings. 

In [6]:
# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DDP = 0.1694 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.8027 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = -0.5444 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DDP = -0.0371 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DDP = 0.2357 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DDP = -0.0371 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.1562
Obtained: DDP = 0.0909 with lambda = 0.1562
----------Iteration #5----------
Testing new Lambda: 0.1719
Obtained: DDP = -0.0371 with lambda = 0.1719
----------Iteration #6----------
Testing new Lambda: 0.1641
Obtained: DDP = -0.0371 with lambda = 0.1641
----------Iteration #7----------
Testing new Lambda: 0.1602
Obtained: DDP = -0.0371 w

SearchFair(reg_beta=0.0001, verbose=True)

To print out the Accuracy and the fairness notions Demographic Parity and Equality of Opportuniy, we define the following function. 

In [7]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

Now lets see, if we obtained a fair classifier with respect to the fairness notions we specified. 

In [8]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 83.0000%
DDP: -3.7102% DEO: -8.2422%
----------Test----------
Accuracy: 80.1250%
DDP: -5.6505% DEO: -13.1601%


The train DDP is small, and SearchFair succeeded to find a fair classifier. The test DDP might or might not be close to 0. This is due to the small number of points which are used to reduce running time for this example notebook. Go ahead and try it with more points!

### Equality of Opportunity

Now we try the same using a more complex rbf kernel, and we try to improve Equality of Opportunity this time. 

In [9]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.1099 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.8573 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.8185 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.0679 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.0176 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = -0.0491 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.1562
Obtained: DEO = -0.0295 with lambda = 0.1562
----------Iteration #5----------
Testing new Lambda: 0.1406
Obtained: DEO = -0.0027 with lambda = 0.1406
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.1406 with fairness -0.0027----------
----------Train----------
Accuracy: 87.9167%
DDP: 4.3470% 

### Cross Validation - GridSearch

It is also possible to use GridSearchCV for the regularization paramter beta, and, if used, the width of the rbf kernel. But running this might take some time.

In [10]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy', refit=True)
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 69.6min finished


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.02631578947368421, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=1)

In [11]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 86.1667%
DDP: -0.5068% DEO: -1.1536%
----------Test----------
Accuracy: 82.5481%
DDP: -0.9058% DEO: -4.7941%


## Adult dataset

In the fairness literature, the adult dataset is a very popular dataset. It contains US census data from 1994, where the class label indicates if the income is higher or lower than 50.000$. The binary sensitive attribute here, is the sex.

In [12]:
# Load Data
x_data, y_data, s_data = get_data.get_adult_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1200
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
ut.print_data_stats(s_data, y_data)

Total data points: 48842
# non-protected examples: 32650
# protected examples: 16192
# non-protected examples in positive class: 9918 (30.4%)
# protected examples in positive class: 1769 (10.9%)


If you want, you can also try SearchFair on Adult. 

In [13]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model_adult = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model_adult,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy')
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 71.6min finished


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.05555555555555555, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=1)

In [14]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 76.1667%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 76.0694%
DDP: 0.0000% DEO: 0.0000%
